In [68]:
from flask import Flask, request, jsonify
import pandas as pdº
import sqlite3


In [69]:
app = Flask(__name__)
app.config['DEBUG'] = False

In [70]:

books = [ 
            {'id': 0, 
            'title': 'A Fire Upon the Deep', 
            'author': 'Vernor Vinge', 
            'first_sentence': 'The coldsleep itself was dreamless.', 
            'published': '1992'},
            
            {'id': 1, 
            'title': 'The Ones Who Walk Away From Omelas', 
            'author': 'Ursula K. Le Guin', 
            'first_sentence': 'With a clamor of bells that set the swallows soaring, the Festival of Summer came to the city Omelas, bright-towered by the sea.', 
            'published': '1973'},

            {'id': 2, 
            'title': 'Dhalgren', 
            'author': 'Samuel R. Delany', 
            'first_sentence': 'to wound the autumnal city.', 
            'published': '1975'} 
        ]
books = pd.DataFrame(books)

In [71]:
books = pd.read_csv('books.csv')
books['published'] = books['published'].astype(int)

In [72]:
@app.route('/', methods=['GET'])
def home():
	return "<h1>La Enciclopedia de Hari Seldon</h1><p>This site is a prototype API for distant reading of science fiction novels.</p>"


# GET Display all the books
@app.route('/api/v1/resources/books/all', methods=['GET'])
def get_all():
    connection = sqlite3.connect('data/books.db')
    cursor = connection.cursor()
    select_books = "SELECT * FROM books"
    result = cursor.execute(select_books).fetchall()
    connection.close()
    return {'books': result}

@app.route('/api/v1/resources/books/1', methods=['GET']) 
def api_1():
    return jsonify(books.iloc[0].to_dict())

@app.route('/api/v1/resources/books/2', methods=['GET']) 
def api_2():
    return jsonify(books.iloc[1].to_dict())

@app.route('/api/v1/resources/books/3', methods=['GET']) 
def api_3():
    return jsonify(books.iloc[2].to_dict())

@app.route('/api/v1/resources/books', methods=['GET']) 
def api_id(): 
 
    if 'id' in request.args: 
        id = int(request.args['id']) 
        results = books[books["id"]==id]
    elif 'author' in request.args: 
        author = request.args['author']
        results = books[books["author"]==author]
    else: 
        return "Error: No id field provided. Please specify an id." 
 

    return jsonify(results.to_dict())

@app.route('/api/v1/resources/books/save', methods=['GET']) 
def api_save(): 
    books.to_csv("books.csv")

    return jsonify('Se ha guardado correctamente')

@app.route('/api/v1/resources/books/query', methods=['GET']) 
def api_query(): 
    first = int(request.args['first']) 
    second = int(request.args['second']) 
    results = books[(books["published"] > first) & (books['published'] <= second)]

    return jsonify(results.to_dict(orient='records'))

app.run(port=8000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [10/Jul/2024 21:14:08] "GET /api/v1/resources/books/all HTTP/1.1" 200 -
